# Notebook requêtant wikidata en SPARQL

Imports

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
import numpy as np
import pandas as pd
import folium
from folium.plugins import MarkerCluster

### 1) Récupération de tous les rois de France

In [191]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?item ?itemLabel (GROUP_CONCAT(DISTINCT ?spouseLabel;separator=", ") AS ?spouses) (GROUP_CONCAT(DISTINCT ?mannerOfDeathLabel;separator=", ") AS ?mannersOfDeath) 
?dateOfBirth ?placeOfBirthLabel ?dateOfDeath ?placeOfDeathLabel ?placeOfBurialLabel ?fatherLabel ?motherLabel ?conflictLabel ?startTime ?endTime

WHERE
{
{?item wdt:P31 wd:Q5 .} 
{?item wdt:P39 wd:Q18384454 .} UNION {?item wdt:P39 wd:Q22923081} UNION {?item wdt:P39 wd:Q3439798}
{?item wdt:P569 ?dateOfBirth} UNION {MINUS {?item wdt:P569 ?dateOfBirth}}
{?item wdt:P19 ?placeOfBirth} UNION {MINUS {?item wdt:P19 ?placeOfBirth}}
{?item wdt:P570 ?dateOfDeath} UNION {MINUS {?item wdt:P570 ?dateOfDeath}}
{?item wdt:P20 ?placeOfDeath} UNION {MINUS {?item wdt:P20 ?placeOfDeath}}
{?item wdt:P509 ?mannerOfDeath} UNION {MINUS {?item wdt:P509 ?mannerOfDeath}}
{?item wdt:P119 ?placeOfBurial} UNION {MINUS {?item wdt:P119 ?placeOfBurial}}
{?item wdt:P22 ?father} UNION {MINUS {?item wdt:P22 ?father}}
{?item wdt:P25 ?mother} UNION {MINUS {?item wdt:P25 ?mother}}
{?item wdt:P26 ?spouse} UNION {MINUS {?item wdt:P26 ?spouse}}
{?item wdt:P607 ?conflict} UNION {MINUS {?item wdt:P607 ?conflict}}
?item p:P39 ?positionHeld.
{?positionHeld pq:P580 ?startTime.} UNION {MINUS {?positionHeld pq:P580 ?startTime.}}
{?positionHeld pq:P582 ?endTime.} UNION {MINUS {?positionHeld pq:P582 ?endTime.}}
  
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr" .
                         ?item rdfs:label ?itemLabel .
                         ?placeOfBirth rdfs:label ?placeOfBirthLabel .
                         ?placeOfDeath rdfs:label ?placeOfDeathLabel .
                         ?mannerOfDeath rdfs:label ?mannerOfDeathLabel .
                         ?placeOfBurial rdfs:label ?placeOfBurialLabel .
                         ?father rdfs:label ?fatherLabel .
                         ?mother rdfs:label ?motherLabel .
                         ?spouse rdfs:label ?spouseLabel .}
}GROUP BY ?item ?itemLabel ?dateOfBirth ?placeOfBirthLabel ?dateOfDeath ?placeOfDeathLabel ?placeOfBurialLabel ?fatherLabel ?motherLabel ?conflictLabel ?startTime ?endTime

""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [192]:
results

{'head': {'vars': ['item',
   'itemLabel',
   'spouses',
   'mannersOfDeath',
   'dateOfBirth',
   'placeOfBirthLabel',
   'dateOfDeath',
   'placeOfDeathLabel',
   'placeOfBurialLabel',
   'fatherLabel',
   'motherLabel',
   'conflictLabel',
   'startTime',
   'endTime']},
 'results': {'bindings': [{'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q134259'},
    'itemLabel': {'xml:lang': 'fr',
     'type': 'literal',
     'value': 'Louis VII de France'},
    'dateOfBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
     'type': 'literal',
     'value': '1120-01-01T00:00:00Z'},
    'placeOfBirthLabel': {'xml:lang': 'fr',
     'type': 'literal',
     'value': 'Paris'},
    'dateOfDeath': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
     'type': 'literal',
     'value': '1180-09-25T00:00:00Z'},
    'placeOfDeathLabel': {'xml:lang': 'fr',
     'type': 'literal',
     'value': 'Paris'},
    'placeOfBurialLabel': {'xml:lang': 'fr',
     'type

In [193]:
rois = []
for r in results['results']['bindings']:
    roi = {}
    if("item" in r):
        roi["wikiID"] = r["item"]["value"].split("http://www.wikidata.org/entity/")[1]
    if("itemLabel" in r):
        roi['nom'] = r['itemLabel']['value']
    if("dateOfBirth" in r):
        roi['dateOfBirth'] = r['dateOfBirth']['value']
    if("placeOfBirthLabel" in r):
        roi['placeOfBirthLabel'] = r['placeOfBirthLabel']['value']
    if("dateOfDeath" in r):
        roi['dateOfDeath'] = r['dateOfDeath']['value']
    if("placeOfDeathLabel" in r):
        roi['placeOfDeathLabel'] = r['placeOfDeathLabel']['value']
    if("mannersOfDeath" in r):
        roi['mannersOfDeath'] = r['mannersOfDeath']['value']
    if("placeOfBurialLabel" in r):
        roi['placeOfBurialLabel'] = r['placeOfBurialLabel']['value']
    if("fatherLabel" in r):
        roi['fatherLabel'] = r['fatherLabel']['value']
    if("motherLabel" in r):
        roi['motherLabel'] = r['motherLabel']['value']
    if("spouses" in r):
        roi['spouses'] = r['spouses']['value']
    if("conflictLabel" in r):
        roi['conflictLabel'] = r['conflictLabel']['value']
    if("startTime" in r):
        roi['startTime'] = r['startTime']['value']
    if("endTime" in r):
        roi['endTime'] = r['endTime']['value']
    rois.append(roi)

In [194]:
df = pd.DataFrame(rois)
df.count()

wikiID                169
nom                   169
dateOfBirth           166
placeOfBirthLabel     148
dateOfDeath           165
placeOfDeathLabel     154
mannersOfDeath        169
placeOfBurialLabel    114
fatherLabel           153
motherLabel           135
spouses               169
startTime              81
endTime                81
dtype: int64

In [195]:
df = df.drop_duplicates()

In [196]:
df.count()

wikiID                169
nom                   169
dateOfBirth           166
placeOfBirthLabel     148
dateOfDeath           165
placeOfDeathLabel     154
mannersOfDeath        169
placeOfBurialLabel    114
fatherLabel           153
motherLabel           135
spouses               169
startTime              81
endTime                81
dtype: int64

In [197]:
pd.options.display.max_rows = 999

In [198]:
df = df.drop_duplicates(subset=["nom"])

In [199]:
df.count()

wikiID                120
nom                   120
dateOfBirth           117
placeOfBirthLabel     102
dateOfDeath           116
placeOfDeathLabel     107
mannersOfDeath        120
placeOfBurialLabel     69
fatherLabel           106
motherLabel            89
spouses               120
startTime              50
endTime                50
dtype: int64

In [200]:
df2 = df

In [201]:
df2.shape

(120, 13)

In [202]:
from datetime import datetime

In [203]:
regex = r'^(-?(?:[1-9][0-9]*)?[0-9]{4})-(1[0-2]|0[1-9])-(3[01]|0[1-9]|[12][0-9])T(2[0-3]|[01][0-9]):([0-5][0-9]):([0-5][0-9])(\.[0-9]+)?(Z|[+-](?:2[0-3]|[01][0-9]):[0-5][0-9])?$'

In [204]:
import dateutil.parser as dp

import re
match_iso8601 = re.compile(regex).match

def validate_iso8601_us(str_val):
    try:
        if match_iso8601( str_val ) is not None:
            return True
    except:
        pass
    return False

In [205]:
validate_iso8601_us("0153-12-23T00:00:00Z")

True

In [206]:
df2['startTime'] = df2['startTime'].map(lambda x: x if validate_iso8601_us(x) else None)
df2['endTime'] = df2['endTime'].map(lambda x: x if validate_iso8601_us(x) else None)


In [207]:
df2['startYear'] = df2['startTime'].map(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S%z").year if validate_iso8601_us(x) else None)
df2['endYear'] = df2['endTime'].map(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S%z").year if validate_iso8601_us(x) else None)


In [208]:
df2

,wikiID,nom,dateOfBirth,placeOfBirthLabel,dateOfDeath,placeOfDeathLabel,mannersOfDeath,placeOfBurialLabel,fatherLabel,motherLabel,spouses,startTime,endTime,startYear,endYear
0,Q134259,Louis VII de France,1120-01-01T00:00:00Z,Paris,1180-09-25T00:00:00Z,Paris,,basilique Saint-Denis,Louis VI de France,Adélaïde de Savoie,"Constance de Castille, Adèle de Champagne, Ali...",1137-08-08T00:00:00Z,1180-09-25T00:00:00Z,1137.0,1180.0
1,Q165284,Louis VIII de France,1187-09-12T00:00:00Z,Paris,1226-11-15T00:00:00Z,Montpensier,dysentérie,basilique Saint-Denis,Philippe II Auguste,Isabelle de Hainaut,Blanche de Castille,1223-07-21T00:00:00Z,1226-11-15T00:00:00Z,1223.0,1226.0
2,Q109223,Childebert IV,0678-01-01T00:00:00Z,t1766657890,0711-04-18T00:00:00Z,t1766657891,,Choisy-au-Bac,Thierry III,Clotilde dite Doda,,0695-01-01T00:00:00Z,0711-01-01T00:00:00Z,695.0,711.0
4,Q133372,Charles VII de France,1403-03-03T00:00:00Z,Paris,1461-07-31T00:00:00Z,Mehun-sur-Yèvre,,basilique Saint-Denis,Charles VI,Isabeau de Bavière,Marie d'Anjou,1422-11-08T00:00:00Z,1461-08-09T00:00:00Z,1422.0,1461.0
5,Q108331,Thierry IV,0712-01-01T00:00:00Z,t1766635643,0737-01-01T00:00:00Z,t1766635644,,basilique Saint-Denis,Dagobert III,NaN,,0721-01-01T00:00:00Z,0737-01-01T00:00:00Z,721.0,737.0
6,Q159575,Hugues Capet,0940-01-01T00:00:00Z,Dourdan,0996-10-29T00:00:00Z,Les Juifs,,basilique Saint-Denis,Hugues le Grand,Hedwige de Saxe,Adélaïde d'Aquitaine,0987-06-08T00:00:00Z,0996-10-29T00:00:00Z,987.0,996.0
7,Q165883,Louis VI de France,1081-12-07T00:00:00Z,Paris,1137-08-08T00:00:00Z,château de la Douye,,basilique Saint-Denis,Philippe Ier de France,Berthe de Hollande,"Lucienne de Rochefort, Adélaïde de Savoie",1108-08-06T00:00:00Z,1137-08-08T00:00:00Z,1108.0,1137.0
8,Q109240,Chilpéric II,0673-01-01T00:00:00Z,t1766643120,0721-02-17T00:00:00Z,Nyon,,Noyon,Childéric II,Bilichilde,,0719-01-01T00:00:00Z,0721-01-01T00:00:00Z,719.0,721.0
12,Q132545,Henri II,1519-04-10T00:00:00Z,Saint-Germain-en-Laye,1559-07-20T00:00:00Z,hôtel des Tournelles,,basilique Saint-Denis,François Ier,Claude de France,Catherine de Médicis,1547-04-10T00:00:00Z,1559-07-20T00:00:00Z,1547.0,1559.0
13,Q134452,Charles VIII,1470-07-09T00:00:00Z,Amboise,1498-04-16T00:00:00Z,Amboise,accident vasculaire cérébral,basilique Saint-Denis,Louis XI,Charlotte de Savoie,Anne de Bretagne,1483-09-17T00:00:00Z,1498-04-25T00:00:00Z,1483.0,1498.0


In [209]:
df2['dateOfBirth'] = df2['dateOfBirth'].map(lambda x: x if validate_iso8601_us(x) else None)
df2['dateOfDeath'] = df2['dateOfDeath'].map(lambda x: x if validate_iso8601_us(x) else None)


In [210]:
df2['birthYear'] = df2['dateOfBirth'].map(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S%z").year if validate_iso8601_us(x) else None)
df2['deathYear'] = df2['dateOfDeath'].map(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S%z").year if validate_iso8601_us(x) else None)

In [211]:
df2.dtypes

wikiID                 object
nom                    object
dateOfBirth            object
placeOfBirthLabel      object
dateOfDeath            object
placeOfDeathLabel      object
mannersOfDeath         object
placeOfBurialLabel     object
fatherLabel            object
motherLabel            object
spouses                object
startTime              object
endTime                object
startYear             float64
endYear               float64
birthYear             float64
deathYear             float64
dtype: object

In [212]:
def which_king(year):
    df_king = df2[df2["startYear"] <= year]
    df_king = df_king[df_king["endYear"] > year]
    return df_king

In [213]:
king = which_king(1609)
king

,wikiID,nom,dateOfBirth,placeOfBirthLabel,dateOfDeath,placeOfDeathLabel,mannersOfDeath,placeOfBurialLabel,fatherLabel,motherLabel,spouses,startTime,endTime,startYear,endYear,birthYear,deathYear
55,Q936976,Henri IV,1553-12-23T00:00:00Z,château de Pau,1610-05-14T00:00:00Z,Paris,blessure par arme blanche,basilique Saint-Denis,Antoine de Bourbon,Jeanne d'Albret,"Marguerite de France, Marie de Médicis",1572-06-19T00:00:00Z,1610-05-14T00:00:00Z,1572.0,1610.0,1553.0,1610.0


### 2) Save in csv

In [4]:
df2.to_csv("../csv/rois-france-avec-dates.csv", index=False)

TypeError: parser_f() got an unexpected keyword argument 'index'

In [ ]:
df2

In [219]:
df2.to_csv("../csv/concat/roi.csv", index=False)

In [215]:
df2.shape

(120, 17)

In [216]:
df2.columns

Index(['wikiID', 'nom', 'dateOfBirth', 'placeOfBirthLabel', 'dateOfDeath',
       'placeOfDeathLabel', 'mannersOfDeath', 'placeOfBurialLabel',
       'fatherLabel', 'motherLabel', 'spouses', 'startTime', 'endTime',
       'startYear', 'endYear', 'birthYear', 'deathYear'],
      dtype='object')

## Récup image si possible

In [5]:
df2 = pd.read_csv("../csv/postgres/roi.csv")

In [6]:
df2

,wikiid,nom,dateofbirth,placeofbirthlabel,dateofdeath,placeofdeathlabel,mannersofdeath,placeofburiallabel,fatherlabel,motherlabel,spouses,starttime,endtime,startyear,endyear,birthyear,deathyear
0,Q134259,Louis VII de France,1120-01-01,Paris,1180-09-25,Paris,NaN,basilique Saint-Denis,Louis VI de France,Adélaïde de Savoie,"Constance de Castille, Adèle de Champagne, Ali...",1137-08-08,1180-09-25,1137.0,1180.0,1120.0,1180.0
1,Q165284,Louis VIII de France,1187-09-12,Paris,1226-11-15,Montpensier,dysentérie,basilique Saint-Denis,Philippe II Auguste,Isabelle de Hainaut,Blanche de Castille,1223-07-21,1226-11-15,1223.0,1226.0,1187.0,1226.0
2,Q109223,Childebert IV,0678-01-01,t1766657890,0711-04-18,t1766657891,NaN,Choisy-au-Bac,Thierry III,Clotilde dite Doda,NaN,0695-01-01,0711-01-01,695.0,711.0,678.0,711.0
3,Q316869,Thierry II,NaN,NaN,NaN,NaN,NaN,NaN,Childebert II,Faileube,Ermenberge,NaN,NaN,595.0,613.0,587.0,613.0
4,Q133372,Charles VII de France,1403-03-03,Paris,1461-07-31,Mehun-sur-Yèvre,NaN,basilique Saint-Denis,Charles VI,Isabeau de Bavière,Marie d'Anjou,1422-11-08,1461-08-09,1422.0,1461.0,1403.0,1461.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Q193658,Louis IV d'Outremer,NaN,Laon,NaN,Reims,chute de cheval,basilique Saint-Remi de Reims,Charles III le Simple,Eadgifu,Gerberge de Saxe,0936-06-19,0954-09-10,936.0,954.0,920.0,954.0
80,Q242832,Lothaire,NaN,Laon,0986-03-02,Laon,NaN,basilique Saint-Remi de Reims,Louis IV d'Outremer,Gerberge de Saxe,Emma d'Italie,0954-09-10,0986-03-02,954.0,986.0,941.0,986.0
81,Q205289,Louis V,NaN,NaN,0987-05-22,Compiègne,accident de chasse,abbaye Saint-Corneille,Lothaire,Emma d'Italie,Adélaïde d'Anjou,0986-03-02,0987-05-21,986.0,987.0,967.0,987.0
82,Q8384,Louis X de France,1289-10-04,Paris,1316-06-05,Vincennes,maladie,basilique Saint-Denis,Philippe IV de France,Jeanne Ire de Navarre,"Marguerite de Bourgogne, Clémence de Hongrie",1314-11-29,1316-06-05,1314.0,1316.0,1289.0,1316.0


In [7]:
from wikidata.client import Client
client = Client()  # doctest: +SKIP

In [9]:
updKings = []
for index, row in df2.iterrows():
    print(row["nom"])
    updKing = row

    entity = client.get(row["wikiid"], load=True)
    try:
        image_prop = client.get('P18')
        image = entity[image_prop]
        updKing["urlImage"] = image.image_url
    except:
        updKing["urlImage"] = ""
    updKings.append(updKing)
df_upd = pd.DataFrame(updKings)
df_upd

Louis VII de France
Louis VIII de France
Childebert IV
Thierry II
Charles VII de France
Thierry IV
Hugues Capet
Louis VI de France
Henri II
Charles VIII
Charles VI
Henri Ier de France
Charles V de France
Philippe VI de France
Charles IV de France
Robert II de France
Charles IX
Dagobert III
Philippe IV de France
Carloman Ier
Clovis Ier
François II
Clovis IV
Childéric II
Dagobert II
Clotaire IV
Louis XIII
Thierry III
Childéric III
François Ier
Jean II de France
Philippe Ier de France
Louis le Pieux
Philippe III de France
Philippe II Auguste
Pépin le Bref
Henri IV
Saint Louis
Louis XII
Louis XI
Louis XIV
Charles X
Philippe V de France
Louis XV
Louis XVIII
Thierry Ier
Théodebert Ier
Childebert Ier
Clotaire Ier
Sigebert Ier
Gontran
Théodebert II
Sigebert II
Caribert II
Sigebert III
Childebert III l'Adopté
Clotaire III
Henri III
Louis XVI
Clovis III
Clodomir
Théodebald
Caribert Ier
Chilpéric Ier
Childebert II
Clotaire II
Dagobert Ier
Clovis II
Chilpéric II
Charlemagne
Charles II le Chauve
Lo

,wikiid,nom,dateofbirth,placeofbirthlabel,dateofdeath,placeofdeathlabel,mannersofdeath,placeofburiallabel,fatherlabel,motherlabel,spouses,starttime,endtime,startyear,endyear,birthyear,deathyear,urlImage
0,Q134259,Louis VII de France,1120-01-01,Paris,1180-09-25,Paris,NaN,basilique Saint-Denis,Louis VI de France,Adélaïde de Savoie,"Constance de Castille, Adèle de Champagne, Ali...",1137-08-08,1180-09-25,1137.0,1180.0,1120.0,1180.0,https://upload.wikimedia.org/wikipedia/commons...
1,Q165284,Louis VIII de France,1187-09-12,Paris,1226-11-15,Montpensier,dysentérie,basilique Saint-Denis,Philippe II Auguste,Isabelle de Hainaut,Blanche de Castille,1223-07-21,1226-11-15,1223.0,1226.0,1187.0,1226.0,https://upload.wikimedia.org/wikipedia/commons...
2,Q109223,Childebert IV,0678-01-01,t1766657890,0711-04-18,t1766657891,NaN,Choisy-au-Bac,Thierry III,Clotilde dite Doda,NaN,0695-01-01,0711-01-01,695.0,711.0,678.0,711.0,https://upload.wikimedia.org/wikipedia/commons...
3,Q316869,Thierry II,NaN,NaN,NaN,NaN,NaN,NaN,Childebert II,Faileube,Ermenberge,NaN,NaN,595.0,613.0,587.0,613.0,https://upload.wikimedia.org/wikipedia/commons...
4,Q133372,Charles VII de France,1403-03-03,Paris,1461-07-31,Mehun-sur-Yèvre,NaN,basilique Saint-Denis,Charles VI,Isabeau de Bavière,Marie d'Anjou,1422-11-08,1461-08-09,1422.0,1461.0,1403.0,1461.0,https://upload.wikimedia.org/wikipedia/commons...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Q193658,Louis IV d'Outremer,NaN,Laon,NaN,Reims,chute de cheval,basilique Saint-Remi de Reims,Charles III le Simple,Eadgifu,Gerberge de Saxe,0936-06-19,0954-09-10,936.0,954.0,920.0,954.0,https://upload.wikimedia.org/wikipedia/commons...
80,Q242832,Lothaire,NaN,Laon,0986-03-02,Laon,NaN,basilique Saint-Remi de Reims,Louis IV d'Outremer,Gerberge de Saxe,Emma d'Italie,0954-09-10,0986-03-02,954.0,986.0,941.0,986.0,https://upload.wikimedia.org/wikipedia/commons...
81,Q205289,Louis V,NaN,NaN,0987-05-22,Compiègne,accident de chasse,abbaye Saint-Corneille,Lothaire,Emma d'Italie,Adélaïde d'Anjou,0986-03-02,0987-05-21,986.0,987.0,967.0,987.0,https://upload.wikimedia.org/wikipedia/commons...
82,Q8384,Louis X de France,1289-10-04,Paris,1316-06-05,Vincennes,maladie,basilique Saint-Denis,Philippe IV de France,Jeanne Ire de Navarre,"Marguerite de Bourgogne, Clémence de Hongrie",1314-11-29,1316-06-05,1314.0,1316.0,1289.0,1316.0,https://upload.wikimedia.org/wikipedia/commons...


In [10]:
1+2

3

In [30]:
df2 = pd.DataFrame(updKings)

In [31]:
df2

,wikiid,nom,dateofbirth,placeofbirthlabel,dateofdeath,placeofdeathlabel,mannersofdeath,placeofburiallabel,fatherlabel,motherlabel,spouses,starttime,endtime,startyear,endyear,birthyear,deathyear,urlImage
0,Q134259,Louis VII de France,1120-01-01,Paris,1180-09-25,Paris,NaN,basilique Saint-Denis,Louis VI de France,Adélaïde de Savoie,"Constance de Castille, Adèle de Champagne, Ali...",1137-08-08,1180-09-25,1137.0,1180.0,1120.0,1180.0,https://upload.wikimedia.org/wikipedia/commons...
1,Q165284,Louis VIII de France,1187-09-12,Paris,1226-11-15,Montpensier,dysentérie,basilique Saint-Denis,Philippe II Auguste,Isabelle de Hainaut,Blanche de Castille,1223-07-21,1226-11-15,1223.0,1226.0,1187.0,1226.0,https://upload.wikimedia.org/wikipedia/commons...
2,Q109223,Childebert IV,0678-01-01,t1766657890,0711-04-18,t1766657891,NaN,Choisy-au-Bac,Thierry III,Clotilde dite Doda,NaN,0695-01-01,0711-01-01,695.0,711.0,678.0,711.0,https://upload.wikimedia.org/wikipedia/commons...
3,Q316869,Thierry II,NaN,NaN,NaN,NaN,NaN,NaN,Childebert II,Faileube,Ermenberge,NaN,NaN,595.0,613.0,587.0,613.0,https://upload.wikimedia.org/wikipedia/commons...
4,Q133372,Charles VII de France,1403-03-03,Paris,1461-07-31,Mehun-sur-Yèvre,NaN,basilique Saint-Denis,Charles VI,Isabeau de Bavière,Marie d'Anjou,1422-11-08,1461-08-09,1422.0,1461.0,1403.0,1461.0,https://upload.wikimedia.org/wikipedia/commons...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Q193658,Louis IV d'Outremer,NaN,Laon,NaN,Reims,chute de cheval,basilique Saint-Remi de Reims,Charles III le Simple,Eadgifu,Gerberge de Saxe,0936-06-19,0954-09-10,936.0,954.0,920.0,954.0,https://upload.wikimedia.org/wikipedia/commons...
80,Q242832,Lothaire,NaN,Laon,0986-03-02,Laon,NaN,basilique Saint-Remi de Reims,Louis IV d'Outremer,Gerberge de Saxe,Emma d'Italie,0954-09-10,0986-03-02,954.0,986.0,941.0,986.0,https://upload.wikimedia.org/wikipedia/commons...
81,Q205289,Louis V,NaN,NaN,0987-05-22,Compiègne,accident de chasse,abbaye Saint-Corneille,Lothaire,Emma d'Italie,Adélaïde d'Anjou,0986-03-02,0987-05-21,986.0,987.0,967.0,987.0,https://upload.wikimedia.org/wikipedia/commons...
82,Q8384,Louis X de France,1289-10-04,Paris,1316-06-05,Vincennes,maladie,basilique Saint-Denis,Philippe IV de France,Jeanne Ire de Navarre,"Marguerite de Bourgogne, Clémence de Hongrie",1314-11-29,1316-06-05,1314.0,1316.0,1289.0,1316.0,https://upload.wikimedia.org/wikipedia/commons...


In [16]:
df2.to_csv("../csv/postgres/roi.csv", index=False)

In [32]:
import urllib.request


In [38]:
updks = []
for index, row in df2.iterrows():
    updk = row
    if(row["urlImage"] != ""):
        newpath = "../frontend/src/images/kings/"+row["wikiid"]+"."+row["urlImage"].split(".")[-1]
        urllib.request.urlretrieve(row["urlImage"], newpath)
        row["urlImage"] = row["wikiid"]+"."+row["urlImage"].split(".")[-1]
    updks.append(updk)
    print(row["nom"])

Louis VII de France
Louis VIII de France
Childebert IV
Thierry II
Charles VII de France
Thierry IV
Hugues Capet
Louis VI de France
Henri II
Charles VIII
Charles VI
Henri Ier de France
Charles V de France
Philippe VI de France
Charles IV de France
Robert II de France
Charles IX
Dagobert III
Philippe IV de France
Carloman Ier
Clovis Ier
François II
Clovis IV
Childéric II
Dagobert II
Clotaire IV
Louis XIII
Thierry III
Childéric III
François Ier
Jean II de France
Philippe Ier de France
Louis le Pieux
Philippe III de France
Philippe II Auguste
Pépin le Bref
Henri IV
Saint Louis
Louis XII
Louis XI
Louis XIV
Charles X
Philippe V de France
Louis XV
Louis XVIII
Thierry Ier
Théodebert Ier
Childebert Ier
Clotaire Ier
Sigebert Ier
Gontran
Théodebert II
Sigebert II
Caribert II
Sigebert III
Childebert III l'Adopté
Clotaire III
Henri III
Louis XVI
Clovis III
Clodomir
Théodebald
Caribert Ier
Chilpéric Ier
Childebert II
Clotaire II
Dagobert Ier
Clovis II
Chilpéric II
Charlemagne
Charles II le Chauve
Lo

In [39]:
dfnew = pd.DataFrame(updks)

In [40]:
dfnew.to_csv("../csv/postgres/roi.csv", index=False)

In [43]:
dfnew[dfnew["urlImage"] == ""]

,wikiid,nom,dateofbirth,placeofbirthlabel,dateofdeath,placeofdeathlabel,mannersofdeath,placeofburiallabel,fatherlabel,motherlabel,spouses,starttime,endtime,startyear,endyear,birthyear,deathyear,urlImage
25,Q316908,Clotaire IV,0685-01-01,t1766639272,0719-01-01,t1766639271,NaN,NaN,Childebert IV,NaN,NaN,0717-01-01,0719-01-01,717.0,719.0,685.0,719.0,
52,Q316888,Sigebert II,0602-01-01,t1776719141,0613-10-13,t1776719140,NaN,NaN,Thierry II,NaN,NaN,NaN,NaN,613.0,613.0,602.0,613.0,
61,Q309915,Théodebald,0534-01-01,NaN,0555-01-01,NaN,NaN,NaN,Théodebert Ier,Deoteria,Waldrade,NaN,NaN,548.0,555.0,534.0,555.0,


In [1]:
dataDict = {'id_event','evenement'}

In [5]:
toto = {}

In [6]:
for key in dataDict:
    print(key)
    toto[key] = "ii"

id_event
evenement


In [7]:
toto

{'id_event': 'ii', 'evenement': 'ii'}

In [16]:
        columns = ['id_event','evenement','startyear','endyear','commentaire']

In [17]:
columns[2]

'startyear'

In [18]:
for key in columns:
            print(key)

id_event
evenement
startyear
endyear
commentaire
